# Graph Coloring Problem

The graph coloring problem is a classic problem of combinatorial optimization. It consists of coloring the vertices of a graph such that no two adjacent vertices share the same color. The objective is to minimize the number of colors used.

<img src="img/graph_coloring.png" alt="Solved Coloring" style="width:400px; height:400px;">

In this notebook, we will show how to model and solve the graph coloring problem using SeaPearl.jl.

## Setup
We will begin by activating the environment and importing the necessary packages.

In [ ]:
using Pkg
Pkg.activate("../../../../")
Pkg.instantiate()
using Revise
using SeaPearl

## Problem formulation

The instances have the following format:
```
4 3
0 1
1 2
1 3
```
The first line contains the number of vertices and the number of edges. Each subsequent line contains the indices of two vertices that are connected by an edge. In the instance displayed above, we have a graph containing 4 vertices and 3 edges. The first edge connects vertices 0 and 1, the second edge connects vertices 1 and 2, and the third edge connects vertices 1 and 3.

## Parsing the Instances

We will now build utilities to parse the instances.

In [3]:
"""
    Edge

    Struct to represent an edge in a graph
"""
struct Edge
    vertex1::Int
    vertex2::Int
end

"""
    ColoringInputData

    Struct to represent the input data of a graph coloring problem
"""
struct ColoringInputData
    edges::Array{Edge}
    numberOfEdges::Int
    numberOfVertices::Int
end

"""parseColoringInput

    Function to parse the input of a graph coloring problem

    # Arguments
    - `filename::String`: path to the file containing the input data

    # Output
    - `::ColoringInputData`: the input data of the graph coloring problem
"""
function parseColoringInput(filename::String)
    raw_input = nothing
    open(filename, "r") do openedFile
        raw_input = read(openedFile, String)
    end
    lines = split(raw_input, '\n')
    firstLine = split(lines[1], ' ')
    numberOfVertices = parse(Int, firstLine[1])
    numberOfEdges = parse(Int, firstLine[2])
    edges = Array{Union{Edge, Nothing}}(nothing, numberOfEdges)
    @assert numberOfEdges + 2 <= length(lines)

    for i in 1:numberOfEdges
        edgeArray = split(lines[i+1], ' ')
        edge = Edge(parse(Int, edgeArray[1])+1, parse(Int, edgeArray[2])+1)
        edges[i] = edge
    end

    return ColoringInputData(edges, numberOfEdges, numberOfVertices)
end

parseColoringInput

In [4]:
gc_instance = parseColoringInput("./data/gc_20_1")

ColoringInputData(Edge[Edge(1, 17), Edge(2, 3), Edge(2, 7), Edge(2, 8), Edge(2, 9), Edge(3, 12), Edge(3, 17), Edge(3, 18), Edge(4, 15), Edge(4, 17)  …  Edge(5, 18), Edge(6, 7), Edge(6, 12), Edge(7, 19), Edge(10, 13), Edge(11, 14), Edge(12, 18), Edge(14, 16), Edge(16, 18), Edge(17, 20)], 23, 20)

## Modeling the Problem

We will now build a model for the problem. The first step is to create a model, implemented in SeaPearl by the `CPModel` struct. `CPModel` needs a trailer to keep track of its current position during search. Therefore, creating a model can be done in the following way:
```julia
trailer = SeaPearl.Trailer()
model = SeaPearl.CPModel(trailer)
```

Next up, we will be using integer variables, implemented as `SeaPearl.IntVar` structs. Creating such variables can be done in the following way:

```julia
SeaPearl.IntVar(minimum_value, maximum_value, variable_name, trailer)
```

We will proceed by creating an array for these variables, one for each vertex in the graph. Then, once variables are created, they need to be tied with a `CPModel` by the way of constraints. For example, to add an Equality constraint ensuring the variable `x` is equal to `1`, we can do the following:

```julia
push!(model.constraints, SeaPearl.EqualConstant(x, 1, trailer))
```

Finally, the model needs an objective in order to optimize. Keep in mind that SeaPearl always minimizes, although this should not be a problem for graph coloring. To add an objective, we can do the following:

```julia
SeaPearl.addVariable!(model, x)
SeaPearl.addObjective!(model, x)
```

Putting it all together, we have the following model.

In [ ]:
trailer = SeaPearl.Trailer()
model = SeaPearl.CPModel(trailer)

### Variable Declaration ###
x = SeaPearl.IntVar[] # Array of variables
for i in 1:gc_instance.numberOfVertices
# =============== Add variables to the model ===============
    new_var = SeaPearl.IntVar(1, gc_instance.numberOfVertices, string(i), trailer)
    push!(x, new_var)
    SeaPearl.addVariable!(model, new_var)
end

### Constraints Declaration ###

# Edge constraints
for edge in gc_instance.edges
    # ==== Define one constraint per edge, guaranteeing their values differ =====
    push!(model.constraints, SeaPearl.NotEqual(x[edge.vertex1], x[edge.vertex2], trailer))
end

# =========== Add symmetry-breaking constraint: first color = 1 ===========
push!(model.constraints, SeaPearl.EqualConstant(x[1], 1, trailer))
# =========== Add symmetry-breaking constraint: first color <= second color ===========
push!(model.constraints, SeaPearl.LessOrEqual(x[1], x[2], trailer))

### Objective ###
# ================ Create one variable for the number of colors, give it the name "numberOfColors" and add it to the model ================
numberOfColors = SeaPearl.IntVar(0, gc_instance.numberOfVertices, "numberOfColors", trailer)
SeaPearl.addVariable!(model, numberOfColors)

# ================ Add the constraint max(x) <= numberOfColors ================
for var in x
    push!(model.constraints, SeaPearl.LessOrEqual(var, numberOfColors, trailer))
end

# ================ Define the objective as the numberOfColors variable ================
SeaPearl.addObjective!(model, numberOfColors)

## Solving the Problem

The model is built; let's solve it! To solve a model, SeaPearl uses the `solve!` function. This function takes a model, a variable selection heuristic and a value selection heuristic. It is possible to build custom heuristics, but for this exercise we will simply be using the Min Domain variable selection heuristic and `SeaPearl.BasicHeuristics`, which selects the maximum value available for each domain. The `solve!` function solves the problem and modifies the model it received to make it store solutions found.

In [6]:
SeaPearl.solve!(model; variableHeuristic=SeaPearl.MinDomainVariableSelection{false}(), valueSelection=SeaPearl.BasicHeuristic())

:Optimal

## Plotting the Solution

Let's visualize the solution obtained

In [ ]:
function get_best_solution(model::SeaPearl.CPModel)
    best_solution = nothing
    best_objective = Inf
    for solution in model.statistics.solutions
        if !isnothing(solution) && solution["numberOfColors"] < best_objective
            best_solution = solution
            best_objective = solution["numberOfColors"]
        end
    end
    return best_solution
end

In [ ]:
best_solution = get_best_solution(model)

In [ ]:
using Graphs, GraphPlot, Plots, Colors

function generate_colors(n::Int)
    colors = [HSV((i - 1) * 360 / n, 1, 1) for i in 1:n]
    return colors
end

# Function to convert ColoringInputData and color dictionary to Graphs.SimpleGraph and color vector
function to_simplegraph_and_colors(input::ColoringInputData, solution::Dict{String,Union{Bool,Int64,Set{Int64}}})
    g = Graphs.Graph(input.numberOfVertices)
    for edge in input.edges
        Graphs.add_edge!(g, edge.vertex1, edge.vertex2)
    end
    colors = generate_colors(solution["numberOfColors"])
    color_vector = [colors[solution[string(v)]] for v in 1:input.numberOfVertices]
    return g, color_vector
end

graph, node_colors = to_simplegraph_and_colors(gc_instance, best_solution)
gplot(graph, nodefillc=node_colors, nodelabel=1:gc_instance.numberOfVertices)
